In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
training_set = tf.keras.utils.image_dataset_from_directory(
    '/content/drive/MyDrive/Final_dataset /New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

Found 5035 files belonging to 33 classes.


In [ ]:
validation_set = tf.keras.utils.image_dataset_from_directory(
    '/content/drive/MyDrive/Final_dataset /New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

Found 15277 files belonging to 33 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

num_classes = len(training_set.class_names)
input_size = (128, 128, 3)
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_size),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(num_classes, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
batch_size = 32
epochs = 10
history = model.fit(
    training_set,
    steps_per_epoch=len(training_set),
    epochs=epochs,
    validation_data=validation_set,
    validation_steps=len(validation_set)
)

Epoch 1/10
158/158 [==============================] - ETA: 0s - loss: 2.2657 - accuracy: 0.8272

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_set)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)

In [ ]:
# Save the model
model.save('/content/drive/MyDrive/Plant_diesease_Datasetfolder/Plant_diesease_Dataset/plant_disease_model.h5')

# Plot training history
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

NameError: name 'model' is not defined

In [ ]:
#import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import numpy as np

# Define the test dataset path
test_dir = '/content/drive/MyDrive/Final_dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/test'

# Load the test dataset
test_dataset = image_dataset_from_directory(
    test_dir,
    image_size=(128, 128),
    batch_size=32,
    label_mode='categorical'
)

In [ ]:
test_dataset = test_dataset.cache().prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Load the trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/Plant_diesease_Datasetfolder/Plant_diesease_Dataset/plant_disease_model.h5')

# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_dataset)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

In [ ]:
predictions = model.predict(test_dataset)
predicted_classes = np.argmax(predictions, axis=1)

In [ ]:
# Plot some sample predictions
def plot_images_with_predictions(dataset, model, class_names, num_images=9):
    plt.figure(figsize=(10, 10))
    for images, labels in dataset.take(1):
        predictions = model.predict(images)
        for i in range(num_images):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            predicted_label = class_names[np.argmax(predictions[i])]
            true_label = class_names[np.argmax(labels[i])]
            plt.title(f"True: {true_label}, Pred: {predicted_label}")
            plt.axis("off")

# Plot sample images with predictions
plot_images_with_predictions(test_dataset, model, test_dataset.class_names)